In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
from SCINet import SCINet

In [2]:
from utils.tools import StandardScaler

In [3]:
model_teacher = SCINet(output_len=24, input_len=96, num_levels=5)
model_student = SCINet(output_len=24, input_len=96, num_levels=1)
model_distill = SCINet(output_len=24, input_len=96, num_levels=1)

In [4]:
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class Dataset_Custom(Dataset):
    def __init__(self,root_path,seq_len,features='MS',target='ActivePower',scale=True,
                 inverse=False,flag0 = 'train'):
        self.seq_len = seq_len
        self.features = features
        self.target = target
        self.scale = scale
        self.inverse = inverse
        self.root_path = root_path
        self.__read_data__(flag = flag0)
        assert flag0 in ['train', 'test', 'val']

        
    def __read_data__(self,flag = 'train'):
        type_map = {'train':0, 'val':1, 'test':2}
        self.set_type = type_map[flag]
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(self.root_path)        
        cols = list(df_raw.columns);
        cols.remove(self.target);
        cols.remove('date')
        df_raw = df_raw[cols + [self.target]]
        num_train = int(len(df_raw)*0.7)
        num_test = int(len(df_raw)*0.2)
        num_vali = len(df_raw) - num_train - num_test
        border1s = [0,num_train - self.seq_len,len(df_raw) - num_test - self.seq_len]
        border2s =  [num_train, num_train + num_vali, len(df_raw)]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]
        df_raw = df_raw[border1:border2]
        if self.features == 'S':
            df_raw = df_raw[[self.target]]
            
        if self.scale:
            self.scaler.fit(df_raw.values)
            data = self.scaler.transform(df_raw.values)
        else:
            data = df_raw.values
        self.df = data
        
        self.mean = self.scaler.mean
        self.std = self.scaler.std
    def __getitem__(self,index):
        s_begin = index
        s_end = s_begin + self.seq_len
        l = s_end
        
        if self.features == 'MS':
            seq = self.df[s_begin:s_end,:-1]
        else:
            seq = self.df[s_begin:s_end]
        label = self.df[l:l+24,-1:]
        
        return seq,label,self.mean,self.std
    def __len__(self):
        return len(self.df) - self.seq_len -24

In [6]:
root = 'D:/dataset/reduced.csv'
seq_len = 96
data_train = Dataset_Custom(root,seq_len,features = 'MS')
data_val = Dataset_Custom(root,seq_len,features = 'MS',flag0 = 'val')
data_test = Dataset_Custom(root,seq_len,features = 'MS', flag0 = 'test')

In [8]:
BATCH_SIZE=36
train_loader = DataLoader(dataset=data_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader=DataLoader(dataset=data_test, batch_size=BATCH_SIZE, shuffle=False)
val_loader=DataLoader(dataset=data_val, batch_size=BATCH_SIZE, shuffle=False)

In [10]:
model_teacher = model_teacher.to(device)
model_student = model_student.to(device)

In [11]:
def get_val_loss(model,val,criterion):
    model = model.cuda()

    val_loader = val
    loss = []
    criterion = criterion
    for m,n,_,_ in val_loader:
        m = m.to(torch.float32).cuda()
        n = n.to(torch.float32).cuda()
        model.eval()
        pred_val = model(m)[:,:,-1:]
        target_val = n
        val_loss = criterion(pred_val,target_val).cpu()
        loss.append(val_loss.item())
        a = np.mean(loss)
    return a

In [12]:
def get_distill_loss(loss_t, loss_s, y, m, v):
    criterion1 = nn.MSELoss()
    criterion2 = nn.SmoothL1Loss()
    mse_t = criterion1(loss_t,y)
    mse_s = criterion1(loss_s,y)
    if mse_t<mse_s + m:
        loss_b = mse_s
    else :
        loss_b = 0
    
    loss_reg = criterion2(loss_s, y) + v * loss_b
    return loss_reg

In [13]:
from tqdm import tqdm
import copy
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_teacher.parameters(),lr = 1e-5)
epochs = 200
train_loss = []
min_epoch = 4
min_val_loss = 2
for epoch in range(epochs):
    model_teacher.train()
    for x,y,_,_ in tqdm(train_loader):
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        x = x.to(device)
        y = y.to(device)
        
        
        pred = model_teacher(x)[:,:,-1:]
        
        optimizer.zero_grad()
        loss = criterion(pred,y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.cpu().item())
        
    val_loss = get_val_loss(model_teacher,val_loader,criterion)
    test_loss = get_val_loss(model_teacher, test_loader, criterion)
    if epoch > min_epoch and val_loss < min_val_loss:
            min_val_loss = val_loss
            best_teacher = copy.deepcopy(model_teacher)      
    print("epoch:{},  train_loss:{}, val_loss:{}, test_loss{}".format(epoch,np.mean(train_loss),min_val_loss, test_loss))

100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:46<00:00,  6.27it/s]


epoch:0,  train_loss:11.505661095950202, val_loss:2, test_loss3.575588359719231


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:38<00:00,  7.48it/s]


epoch:1,  train_loss:10.023771889021306, val_loss:2, test_loss2.8966360148929415


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:39<00:00,  7.44it/s]


...

epoch:102,  train_loss:1.1352907492967275, val_loss:0.4932948350019398, test_loss0.3973001364973329


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:39<00:00,  7.44it/s]


epoch:103,  train_loss:1.128202180686118, val_loss:0.4926398095807859, test_loss0.3947800190986267


 28%|██████████████████████▌                                                          | 81/291 [00:11<00:28,  7.31it/s]


KeyboardInterrupt: 

In [16]:
torch.save(best_teacher,'D:/model/5l_teacher.pkl') 

In [16]:
from tqdm import tqdm
import copy
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_student.parameters(),lr = 1e-5)
epochs = 200
train_loss = []
min_epoch = 4
min_val_loss = 1
model = model_student
for epoch in range(epochs):
    model.train()
    for x,y,_,_ in tqdm(train_loader):
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        x = x.to(device)
        y = y.to(device)
        
        
        pred = model(x)[:,:,-1:]
        optimizer.zero_grad()
        loss = criterion(pred,y)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.cpu().item())
        
    val_loss = get_val_loss(model,val_loader,criterion)
    
    if epoch > min_epoch and val_loss < min_val_loss:
            min_val_loss = val_loss
            best_student = copy.deepcopy(model)   
            test_loss = get_val_loss(best_student, test_loader, criterion)
    print("epoch:{},  train_loss:{}, val_loss:{}, test_loss:{}".format(epoch,np.mean(train_loss),min_val_loss, test_loss))

100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:03<00:00, 95.79it/s]


epoch:0,  train_loss:2.0684544835303655, val_loss:1, test_loss:0.39022810774899663


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:02<00:00, 97.62it/s]


epoch:1,  train_loss:1.874909710228648, val_loss:1, test_loss:0.39022810774899663


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:02<00:00, 97.88it/s]


...

epoch:197,  train_loss:0.5048946012941017, val_loss:0.5526930365179267, test_loss:0.44056900776922703


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:02<00:00, 97.59it/s]


epoch:198,  train_loss:0.5042449688758692, val_loss:0.5526930365179267, test_loss:0.44056900776922703


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:03<00:00, 96.81it/s]


epoch:199,  train_loss:0.5036096251650979, val_loss:0.5526930365179267, test_loss:0.44056900776922703


In [17]:
torch.save(best_student, "D:/model/best_student.pkl")

In [13]:
model_teacher_best = torch.load("D:/model/5l_teacher.pkl")

In [16]:
from tqdm import tqdm
import copy
model2 = model_distill
model2 = model2.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_distill.parameters(),lr = 1e-4)
epochs = 50
train_loss = []
min_epoch = 2
min_val_loss = 2
for epoch in range(epochs):
    model2.train()
    for x,y,_,_ in tqdm(train_loader):
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        x = x.to(device)
        y = y.to(device)
        
        
        pred = model2(x)[:,:,-1:]
        pred_teacher = model_teacher_best(x)[:,:,-1:]
        optimizer.zero_grad()
        loss = get_distill_loss(pred_teacher, pred, y, m=0.03, v=0.8)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.cpu().item())
        
    val_loss = get_val_loss(model2,val_loader,criterion)
    test_loss = get_val_loss(model2, test_loader, criterion)
    if epoch > min_epoch and val_loss < min_val_loss:
            min_val_loss = val_loss
            best_model = copy.deepcopy(model2) 
            
    print("epoch:{},  train_loss:{}, val_loss:{}, test_loss{}".format(epoch,np.mean(train_loss),min_val_loss, test_loss))

100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:12<00:00, 24.13it/s]


epoch:0,  train_loss:0.3941637346732248, val_loss:2, test_loss0.4650915049992147


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:11<00:00, 24.32it/s]


epoch:1,  train_loss:0.3911643190328608, val_loss:2, test_loss0.458406928971055


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:11<00:00, 24.29it/s]


...

epoch:7,  train_loss:0.3866593455863777, val_loss:0.6081680225297099, test_loss0.46458109128954156


100%|████████████████████████████████████████████████████████████████████████████████| 291/291 [00:12<00:00, 23.99it/s]


epoch:8,  train_loss:0.3857540167603569, val_loss:0.6081680225297099, test_loss0.46838092320554314


  7%|█████▌                                                                           | 20/291 [00:00<00:12, 22.32it/s]


KeyboardInterrupt: 

In [20]:
torch.save(best_model, "D:/model/best_model.pkl")